<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning_basic/blob/master/ML_RE4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ML8~ML9

**1. 역전파**

**각 가중치 매개변수에 대한 손실함수의 기울기**가 핵심임은 분명하다.

numerical_gradient는 명확하게 위의 개념이 도입되지만, 

gradient는 엄밀히 말하면 각 가중치 매개변수에 대한 (예측값- 정답레이블)의 기울기이다.

아무튼 두 경우 모두 모델의 예측값과 정답 레이블의 오차를 줄이는데에 같은 뜻을 둔다.

역전파의 경우 , 연쇄법칙 덕분에 구현이 가능하다. 

ex) dz/dx = dz/dt * dt/dx 

**기본적으로 역전파의 개념은 일단 입력 신호에 그 노드의 편미분을 곱하는 것이다**

▶덧셈 노드 

    ex) z  = x + y 
    dz/dx = dz/dy = 1 이므로 
    역전파의 개념으로 보면 입력으로 들어온 dL/dz를 그대로 다음 노드로 보낸다

▶곱셈 노드

    ex) z = x * y
    dz/dx = y, dz/dy = x이므로
    상류의 값에 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다
    그렇기 떄문에 순전파의 입력신호의 값을 변수에 저장할 필요가 있다. 

In [2]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  def forward(self,x,y):
    self.x = x
    self.y = y
    out = x * y
    return out
  def backward(self,dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx,dy
class AddLayer:
  def __init__(self):
    pass
  def forward(self,x,y):
    out = x + y
    return out
  def backward(self,dout):
    dx = dout * 1
    dy = dout * 1
    return dx,dy

지금까지는 각 계층이 단순 더하기나 곱하기 였지만 이제는 활성화 함수가 도입된다.

(물론 각 활성화 함수를 세분화하면 더하기와 곱하기 같은 기본 단위로 나눌 수 있긴 하다)

**2. 활성화함수**

⚾Relu

In [ ]:
import numpy as np 
class Relu:
  def __init__(self):
    self.mask = None 
  def forward(self,x):
    self.mask = (x<= 0)
    out = x.copy()
    out[self.mask] = 0
    return out
  def backward(self,dout):
    dout[self.mask] = 0
    dx = dout
    return dx
#같은 흐름의 x와 dout는 그 운명을 같이한다
#Relu는 쉽게 __/ 모양이다. 

⚾Sigmoid

In [3]:
class Sigmoid:
  def __init__(self):
    self.out = None
  def forward(self,x):
    out = 1/(1+np.exp(-x))
    self.out = out
    return out
  def backward(self,dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

⚾Affine




In [3]:
class Affine:
  def __init__(self,W,b):
    #self로 저장한 변수들은 추후에 다 쓰인다
    self.W = W 
    self.b = b 
    self.x = None 
    self.dw = None 
    self.db = None 
  def forward(self, x):
    self.x = x
    out = np.dot(x,self.W) + self.b
    return out
  def backward(self,dout):
    #행렬곱 affine 역전파의 공식이다.
    #행렬곱의 특성상 행렬에 대응하는 차원의 원소 수가 일치하도록 곱을 조립하는 것이 필요 (전치행렬사용)
    dx  = np.dot(dout, self.W.T) 
    self.dW = np.dot(self.x.T, dout) 
    self.db = np.sum(dout, axis=0) 
    #배치크기가 1일경우, 즉 배치가 아닐 경우도 모두 통합시킨 코드.
    return dx

⚡ 위 코드에서 self. db = np.sum(dout, axis=0)인 이유

기본적으로 순전파의 흐름이 
Y = np.dot(X,W) + B이고,

계산 그래프의 원리를 잘 떠올려 역전파를 구해보면

dB, 즉 편항의 기울기는 1차원형태의 배열로 저장되어져야 하고,

덧셈 노드에 의해 dY의 값이 그대로 dB로 흘러들어옴을 알 수 있다.

그렇기 때문에 2차원 형태였던 dY를 1차원으로 바꾸어주어야 하므로, 저러한 코드를 사용하였다



In [2]:
import numpy as np
dY = np.array([[1,2,3],[4,5,6]])
dB = np.sum(dY,axis=0)
dB

array([5, 7, 9])

이렇게 각 데이터 [1,2,3] 과 [4,5,6]이 합쳐져 하나의 데이터 형태로 바뀌는 모습

다시 말해, 2차원 데이터가 1차원 데이터로 변환된 모습

사실, 위 같은 코드는 **배치형태일때와 배치형태가 아닐때(데이터가 1개일 때) 
모든 경우를 고려해 통합한 코드**이다. (앞으로도 자주 사용.)